In [1]:
#Definindo finais de semana
mes = "05"
lista_fds = ['04','05','11','12','18','19','25','26']

In [2]:
# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#definindo quantidade de linhas e colunas visiveis
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Manipulação de Strings
import unidecode # => unidecode(string)
import re # Regex

# Repositorio - Diretorio
import os
import shutil

In [3]:
#######################-FUNÇÕES
def count_freq(data_frame, coluna):
    freq = df[coluna].value_counts()
    perc = df[coluna].value_counts(normalize = True)*100
    tabela = pd.DataFrame({"Freq":freq, "Perc":perc})
    return display(tabela)
        
def tabela_cont(linha, coluna):
    a = df.groupby([linha, coluna]).size().unstack()
    for i in a.columns:
        a[i].fillna(0, inplace = True)
        a[i] = a[i].apply(lambda x: int(x))
    return a
    print('='*100)

def tabela_cont_percent(linha, coluna, opcao):
    result = tabela_cont(linha, coluna)
    if opcao == 'Linha':
        return round(result.div(result.sum(axis=1), axis=0) * 100, 2)
    elif opcao == 'Coluna':
        return round(result.div(result.sum(), axis=1) * 100, 2)
    else:
        return 'Opção invalida'

In [4]:
def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        len_antigo = len(dataframe)
        
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe = dataframe.drop_duplicates(subset=['chat'])
        
        print(len(df1), " += ", len_antigo, "=>", len(dataframe))
        
        dataframe.to_excel(nome_arquivo, index=False)
    
    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print(len(dataframe)) 

In [5]:
def read_file(file_path):
    # 1. Ler o arquivo txt
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    
    # 2. Remover todas as linhas que contêm "Mensagem apagada"
    lines = [line for line in lines if "Mensagem apagada" not in line]
    
    # 3. Corrigir as mensagens que estão em várias linhas para que fiquem em uma única linha
    corrected_lines = []
    current_message = ""
    
    for line in lines:
        # Se a linha começa com o padrão de data e hora, ela é uma nova mensagem
        if re.match(r"^\[\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}\]", line):
            if current_message:  # Se há uma mensagem anterior, adicione-a à lista
                corrected_lines.append(current_message)
            current_message = line.strip()  # Comece uma nova mensagem
        else:
            current_message += " " + line.strip()  # Adicione o conteúdo da linha atual à mensagem em progresso
    
    # Adiciona a última mensagem, se houver
    if current_message:
        corrected_lines.append(current_message)
    
    # Salvando arquivo txt
    with open('processed.txt', "w", encoding="utf-8") as file:
        for line in corrected_lines:
            file.write(line + "\n")
    
    # Lendo arquivo txt e transformando em dataframe cada mensagem
    with open('processed.txt', 'r', encoding='utf-8') as file:
        content = file.read().splitlines()
    
    return content

In [6]:
df = pd.DataFrame({'chat': read_file('_chat.txt') })
#df['chat'] = df['chat'].apply(lambda x: x.replace('/05/2024, ','/05/2024 '))
df['controle'] = np.nan
df = df[['controle','chat']]
df_prob = pd.DataFrame(columns = ['controle','chat'])

print(len(df))
display(df.head())
display(df_prob)

261


,controle,chat
0,NaN,[29/05/2024 15:05:22] Thales Luvisotto Neder: ...
1,NaN,[29/05/2024 15:48:25] Juliano Moisés Tobal: 29...
2,NaN,[29/05/2024 15:52:53] Thiago Barreto Marques d...
3,NaN,[29/05/2024 16:00:28] Amanda Kiyoko Souza Koss...
4,NaN,[29/05/2024 16:11:50] Helena Araujo Damasceno:...


,controle,chat


In [7]:
def sample_prob(nivel):
    '''
    data = df_prob[df_prob['controle'] == nivel]
    tamanho = len(data)
    try:
        if tamanho == 0:
            data_exibir = str('0 itens do prob' + nivel)
        if tamanho == 1:
            data_exibir = data['mensagem'].sample(1)
        if tamanho == 2:
            data_exibir = data['mensagem'].sample(2)
        if tamanho > 5:
            data_exibir = data['mensagem'].sample(5)
    except:
        data_exibir = 'erro'
    '''
    return 'nada'

# DIA QUE FOI ENVIADA A MENSAGEM

In [8]:
dia = []
for i in range(0, len(df['chat'])):
    dia.append(df['chat'][i][1:11]) #pegando exatamente aonde parte da data esta
df['dia'] = dia
#print(sorted(df['dia'].unique()))
print("Quantidade de mensagens por dia:")
count_freq(df, 'dia')

Quantidade de mensagens por dia:


,Freq,Perc
dia,,
30/05/2024,117,44.827586
31/05/2024,89,34.099617
29/05/2024,55,21.072797


# NOME DO ANESTESISTA

In [9]:
df['nome'] = np.nan
count = 0
for i in range(0, len(df['chat'])):
    loc = df['chat'][i].find(']')+1 #digito data
    loc2 = df['chat'][i][loc+1:].find(':')+1 #digito nome
    nome = df['chat'][i][loc+1:loc+loc2]
    nome = nome.upper().strip().replace("  ", " ")
    nome = unidecode.unidecode(nome)
    df['nome'][i] = nome
    count += 1
    #print(count, "->", unidecode.unidecode(chat[i][loc+1:loc+loc2].upper()))

print("Profissionais que mandaram mensagem:")
sorted(df['nome'].unique())

Profissionais que mandaram mensagem:


['ALESSANDRE LUIZ BRAGA JUNIOR',
 'ALEXANDRE BERNARDINO',
 'AMANDA KIYOKO SOUZA KOSSAKA',
 'ANA CLARA MOURAO BARRETO',
 'ANA JUNG DOCKO',
 'ANDRE DA COSTA AGUIAR VENTURA',
 'ANDRE LUIS XAVIER FRANCO',
 'ARTHUR OLIVEIRA FACCHINI',
 'ARTHUR RODRIGUES TORRELIO',
 'BEATRIZ ANTIPOU DOS SANTOS',
 'BRIAN SOSSAI PACHECO',
 'BRUCE YE MAN CHOW',
 'BRUNO FERRAZ DE CAMPOS BARONE',
 'BRUNO STORNIOLI TAGUCHI LEMOS',
 'CAIO TULIO DUARTE',
 'CAMILA CHIARANTANO HARRISON',
 'CARLOS AUGUSTO TREVISAN',
 'CARLOS EDUARDO CORREA GRILO',
 'CARLOS EDUARDO GONDIM OLIVEIRA',
 'CAROLINE FIRMINO LEMES DA SILVA',
 'DANIEL MASUZAKI WONG',
 'DANIELLEN LINS LOURENCO',
 'DARLAN JUNIOR SCHNEIDER',
 'DENIO LOPES DE ALMEIDA',
 'DIOGO HENRIQUE PEREIRA MATTOS',
 'EDUARDO ARBACHE BEZERRA',
 'EMERSON DOMINGOS DA COSTA',
 'ERICK OLIVEIRA LIMA GOMES',
 'ESTANRLEY BARCELOS PINTO',
 'EVERTON SIDNEY DA CONCEICAO CARVALHO',
 'FABIANA BARBOSA BASTOS',
 'FABIANA SOARES GAMA',
 'FABIO YUJI SUGUITA',
 'FELIPE DE OLIVEIRA NUNES',
 'FERN

# MENSAGENS ESCREVIDAS

In [10]:
mensagem = []
for i in range(0, len(df)):
    digito_data = df['chat'][i].find(']') #aonde acaba a data
    digito_nome = df['chat'][i][digito_data+1:].find(':')
    mensagem_escrevida = df['chat'][i][digito_data+digito_nome+3:]
    mensagem.append(mensagem_escrevida)
    
df['mensagem'] = mensagem
df['mensagem'] = df['mensagem'].apply(lambda x: x.upper() )
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('-',' - '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('(',' ( '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace(')',' ) '))

print('Mensagens Enviadas:')
df['mensagem'].sample(3)

Mensagens Enviadas:


174    31/05 JABAQUARA  -  ERICK ENTRADA 05:20
141    30/05 MORUMBI  -  G.MENEZES SAÍDA 15:45
187         31/05 SANTA  -  CLARA ENTRADA 6:00
Name: mensagem, dtype: object

# DATA DO PLANTÃO

In [11]:
df['data'] = np.nan

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'DATA_DO_PLANTÃO'

#salvando
df_prob = df[df['controle'] == 'DATA_DO_PLANTÃO']
print("data probs:", len(df_prob))
sample_prob('DATA_DO_PLANTÃO')

df = df[df['controle'] != 'DATA_DO_PLANTÃO']
print("datas ok:", len(df)) 

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

count_freq(df, 'data')
#print("Dias mensionados nas mensagens:")
#print(sorted(df['data'].unique()))

data probs: 1
datas ok: 260


,Freq,Perc
data,,
30/05,116,44.615385
31/05,85,32.692308
29/05,55,21.153846
29/06,1,0.384615
30/04,1,0.384615
26/05,1,0.384615
31/5,1,0.384615


# CAPTURANDO UNIDADE

In [12]:
df['unidade'] = ''
df['Q_unidades'] = ''

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    
    #Unidades escrita com a bunda
    lista_avi = ["CENTRAL","AVICENA","AVICENNA","AVICCENNA"]
    for i_avi in lista_avi:
        mensagem = mensagem.replace(i_avi, 'AVICCENA')
        
    mensagem = mensagem.replace('Sta Isabel', 'SANTA')
    
    lista_unidades = re.findall(r'ALPHA|ANALIA|AVICCENA|ITAIM|JABA|MATERNIDADE|MORUMBI|OSASCO|VILLA|VILA|VNS|SANTA', mensagem)
    quantidade_unidades = len(lista_unidades)
    
    df['Q_unidades'][i] = quantidade_unidades
    
    if quantidade_unidades == 1: #capturando apenas com uma unidade
        df['unidade'][i] = lista_unidades[0].replace('  ',' ').upper().strip()
    elif quantidade_unidades > 1: #com mais de uma unidade
        df['unidade'][i] = lista_unidades
        df['controle'][i] = 'DESLOCADO'
    else: #sem nenhuma unidade
        df['controle'][i] = 'UNIDADES'
        
print("quantidades de unidades presentes nas mensagens:")
count_freq(df, 'Q_unidades')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'UNIDADES']])
print("unidades probs:", len(df_prob))
sample_prob('UNIDADES')

df = df[df['controle'] != 'UNIDADES']

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
#print("unidade ok:", len(df)) 
#print("unidade ok deslocado:", len(df[df['controle'] == 'DESLOCADO']))

quantidades de unidades presentes nas mensagens:


,Freq,Perc
Q_unidades,,
1,249,95.769231
2,11,4.230769


unidades probs: 1


# RETIRANDO REGISTROS DE DESLOCADOS, DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE

In [13]:
#preparando para reconhecer corretamente
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCADA','DESLOCADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCAMENTO','DESLOCADO'))

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DESLOCADO', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes > 0:
        df['controle'][i] = 'DESLOCADO'
        
count_freq(df, 'controle')
# OBS: Plantões de deslocado não são mais excluido

,Freq,Perc
controle,,
DESLOCADO,13,100.0


In [14]:
#RETIRANDO REGISTROS DE DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE OU DISPENSA
df['controle2'] = np.nan
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSADA','DISPENSADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSA ','DISPENSADO'))

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DISPENSADO|SAUDE|DISTANCIA|D[0-9]', mensagem)
    quantidade_observacoes = len(observacoes)

    if quantidade_observacoes >= 1:
        df['controle'][i] = 'DISP_SAUDE_DIST'
        if quantidade_observacoes == 1:
            df['controle2'][i] = observacoes[0]
        
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'DISP_SAUDE_DIST']])
print("unidades probs:", len(df_prob))
sample_prob('DISP_SAUDE_DIST')

df = df[df['controle'] != 'DISP_SAUDE_DIST']
print("unidade ok:", len(df)) 

print(df_prob['mensagem'][df_prob['controle'] == 'DISP_SAUDE_DIST'].sample(2))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DISP_SAUDE_DIST,34,73.913043
DESLOCADO,12,26.086957


unidades probs: 35
unidade ok: 226
204      31/05 VILLA  -  RENATO DISPENSADO
258    31/05 JABAQUARA FACCHINI DISPENSADO
Name: mensagem, dtype: object


# VERIFICANDO SE É EXTRA

In [15]:
df["extra"] = np.nan

for i in range(0, len(df)):
    if 'EXTRA' in df['mensagem'][i].upper(): #se conter a palavra "extra"
        df['extra'][i] = 'extra' #marque na coluna extra como extra
        
print("Quantidade de registros Extra - Voluntario:")
count_freq(df, 'extra')      

Quantidade de registros Extra - Voluntario:


,Freq,Perc
extra,,


# Tipo entrada e saida:

In [16]:
#Indentificando se é um registro de entrada ou saida
df['tipo'] = np.nan

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    entradas_saidas = re.findall(r'ENTRADA|SAIDA', mensagem)
    quantidade_entradas_saidas = len(entradas_saidas)
    
    if quantidade_entradas_saidas != 1: #capturando apenas com uma informação escrita
        df['controle'][i] = 'SEM_ENTRADA_OU_SAIDA'
    else: #restante com mais de uma informação
        df['tipo'][i] = entradas_saidas[0]
        
count_freq(df, 'tipo')
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'SEM_ENTRADA_OU_SAIDA']])
print("unidades probs:", len(df_prob))
sample_prob('SEM_ENTRADA_OU_SAIDA')

df = df[df['controle'] != 'SEM_ENTRADA_OU_SAIDA']
print("unidade ok:", len(df)) 

#print(df_prob['mensagem'][df_prob['controle'] == 'SEM_ENTRADA_OU_SAIDA'].sample(1))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

,Freq,Perc
tipo,,
SAIDA,161,71.555556
ENTRADA,64,28.444444


,Freq,Perc
controle,,
DESLOCADO,12,92.307692
SEM_ENTRADA_OU_SAIDA,1,7.692308


unidades probs: 36
unidade ok: 225
ok: 225


# Data:

In [17]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = np.nan

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio

In [18]:
#arrumando data d/mm (6/10, não 06/10)
for i in range(0, len(df)):
    if df['data'][i][1] == '/':
        df['data'][i] = '0'+df['data'][i]
df['data'].unique()

array(['29/05', '29/06', '30/05', '30/04', '26/05', '31/05'], dtype=object)

In [19]:
#somando informações restantes na data: dd/MM/yyyy
for i in range(0, len(df)):
    df['data'][i] = df['data'][i][0:2] + "/"+ mes +"/2024" 
    
count_freq(df, 'data')

,Freq,Perc
data,,
30/05/2024,99,44.000000
31/05/2024,69,30.666667
29/05/2024,56,24.888889
26/05/2024,1,0.444444


In [20]:
df['sem_ou_fds'] = 'semana'

for i in lista_fds:
    datinha = i+"/"+mes+"/2024"
    df['sem_ou_fds'][df['data'] == datinha] = 'fds'

count_freq(df, 'sem_ou_fds')

,Freq,Perc
sem_ou_fds,,
semana,224,99.555556
fds,1,0.444444


In [21]:
print(df.groupby(['data', 'sem_ou_fds']).size().unstack())

sem_ou_fds  fds  semana
data                   
26/05/2024  1.0     NaN
29/05/2024  NaN    56.0
30/05/2024  NaN    99.0
31/05/2024  NaN    69.0


In [22]:
#df['data'].iloc[180] = '26/09/2022'

# Hora:

In [23]:
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

In [24]:
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas == 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'HORA'
        
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'HORA']])
print("probs:", len(df_prob))
sample_prob('HORA')

df = df[df['controle'] != 'HORA']
print("ok:", len(df)) 

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,12,100.0


probs: 36
ok: 225


In [25]:
#adicionando 0 no primeiro algorismo dos valores h:mm
for i in range(0, len(df)):
    if df['hora'][i][1] == ":":
        df['hora'][i] = "0"+df['hora'][i]
        
df['hora'].unique()

array(['15:05', '14:00', '16:00', '16:10', '16:15', '17:00', '17:06',
       '16:30', '17:08', '17:15', '17:10', '09:05', '05:45', '05:20',
       '18:45', '19:07', '19:10', '13:00', '19:05', '19:20', '19:30',
       '19:35', '19:40', '19:50', '14:10', '09:00', '20:00', '19:25',
       '20:25', '20:30', '21:10', '21:00', '21:20', '21:15', '21:40',
       '21:50', '22:05', '22:00', '04:30', '05:00', '05:35', '05:40',
       '06:00', '20:50', '06:10', '05:50', '06:30', '05:30', '07:50',
       '04:45', '08:40', '09:50', '10:20', '10:30', '10:44', '11:00',
       '10:55', '11:10', '11:40', '11:30', '11:45', '12:15', '11:55',
       '12:55', '13:15', '13:20', '12:00', '13:10', '12:45', '13:40',
       '10:40', '14:30', '14:40', '14:50', '15:15', '15:20', '15:26',
       '15:10', '15:45', '15:50', '16:40', '16:45', '16:25', '13:45',
       '19:45', '20:15', '20:20', '10:10', '20:45', '05:15', '07:00',
       '07:40', '09:10', '09:20', '09:47', '10:06', '10:45', '10:50',
       '11:35', '12:

# Hora final:

In [26]:
#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

In [27]:
df['hora'].unique()

array(['15:05', '14:00', '16:00', '16:10', '16:15', '17:00', '17:06',
       '16:30', '17:08', '17:15', '17:10', '09:05', '05:45', '05:20',
       '18:45', '19:07', '19:10', '13:00', '19:05', '19:20', '19:30',
       '19:35', '19:40', '19:50', '14:10', '09:00', '20:00', '19:25',
       '20:25', '20:30', '21:10', '21:00', '21:20', '21:15', '21:40',
       '21:50', '22:05', '22:00', '04:30', '05:00', '05:35', '05:40',
       '06:00', '20:50', '06:10', '05:50', '06:30', '05:30', '07:50',
       '04:45', '08:40', '09:50', '10:20', '10:30', '10:44', '11:00',
       '10:55', '11:10', '11:40', '11:30', '11:45', '12:15', '11:55',
       '12:55', '13:15', '13:20', '12:00', '13:10', '12:45', '13:40',
       '10:40', '14:30', '14:40', '14:50', '15:15', '15:20', '15:26',
       '15:10', '15:45', '15:50', '16:40', '16:45', '16:25', '13:45',
       '19:45', '20:15', '20:20', '10:10', '20:45', '05:15', '07:00',
       '07:40', '09:10', '09:20', '09:47', '10:06', '10:45', '10:50',
       '11:35', '12:

In [28]:
#criando uma coluna datetime para fazer calculos de horas e minutos
df['hora2'] = df['hora'].apply(lambda x: datetime.strptime(x, '%H:%M'))

#verificando se esta tudo certo
print(df['hora2'].dt.year.unique())
print(df['hora2'].dt.month.unique())
print(df['hora2'].dt.day.unique())
print(df['hora2'].dt.hour.unique())

for i in range(0, len(df)):
    minuto = int(df['hora'][i][3:])
    
    if minuto != 0 and minuto != 30:
        if minuto < 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (0) subtraindo quantos minutos tem entre o momento e HH:00
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-minuto))[11:-3]
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (30) acrecentando quantos minutos faltam para HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(30-minuto)))[11:-3]
            else:
                df['controle'][i] = 'Hora_final'
                
        elif minuto > 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (30) subtraindo quando minutos tem entre o momento e HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-(minuto-30)))[11:-3]   
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (60) acrecentando quantos minutos faltam para HH:60
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(60-minuto)))[11:-3]            
            else:
                df['controle'][i] = 'Hora_final'
        else:
            df['controle'][i] = 'Hora_final'
            
    elif minuto == 0 or minuto == 30:
        df['hora'][i] = str(df['hora2'][i])[11:-3] 
        
    else:
        df['controle'][i] = 'Hora_final'

df['hora'].unique()

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'Hora_final']])
print("probs:", len(df_prob))
sample_prob('Hora_final')

df = df[df['controle'] != 'Hora_final']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

[1900]
[1]
[1]
[15 14 16 17  9  5 18 19 13 20 21 22  4  6  7  8 10 11 12]
probs: 36
ok: 225


# Setor:

In [29]:
#reconhendo em qual periodo do dia aconteceu o plantão
df['ENTRADA'] = ''
df['SAIDA'] = ''

df['PERIODO'] = ''

for i in range(0, len(df)):
    minuto = int(df['hora'][i][:2])
    
    if df['tipo'][i] == 'ENTRADA':
        df['ENTRADA'][i] = df['hora'][i]
        if minuto < 7: #periodo 6h
            df['PERIODO'][i] = '6h'
            df['SAIDA'][i] = '07:00'
        elif minuto >= 7 and minuto < 13: #periodo manha
            df['PERIODO'][i] = 'manha'
            df['SAIDA'][i] = '13:00'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            df['PERIODO'][i] = 'tarde'
            df['SAIDA'][i] = '19:00'
        else:
            df['controle'][i] = 'HORARIOS'
    elif df['tipo'][i] == 'SAIDA':
        df['SAIDA'][i] = df['hora'][i]
        if minuto >= 7 and minuto < 13: #periodo manha
            df['ENTRADA'][i] = '07:00'
            df['PERIODO'][i] = 'manha'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            if df['hora'][i] == '13:00':
                df['SAIDA'][i] = '13:00'
                df['ENTRADA'][i] = '07:00'
                df['PERIODO'][i] = 'manha'
            else:
                df['ENTRADA'][i] = '13:00'
                df['PERIODO'][i] = 'tarde'
        elif minuto >= 19: #periodo noturno
            df['ENTRADA'][i] = '19:00'
            df['PERIODO'][i] = 'noturno saida'
        else:
            df['controle'][i] = 'HORARIOS'
    else:
        df['controle'][i] = 'HORARIOS'
        
count_freq(df, 'PERIODO')
count_freq(df, 'controle')

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'HORARIOS']])
print("probs:", len(df_prob))
sample_prob('HORARIOS')

df = df[df['controle'] != 'HORARIOS']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
PERIODO,,
tarde,71,31.555556
manha,56,24.888889
6h,54,24.000000
noturno saida,44,19.555556


,Freq,Perc
controle,,
DESLOCADO,12,100.0


probs: 36
ok: 225


# DURAÇÃO DO PLANTÃO

In [30]:
df['DURAÇÃO'] = ''

for i in range(0, len(df)):
    diferenca_tempo = datetime.strptime(df['SAIDA'][i], '%H:%M') - datetime.strptime(df['ENTRADA'][i], '%H:%M')
    
    #diferenca_tempo vem em formato estranho, tem-se que transformar em segundos antes
    diferenca_tempo_s = diferenca_tempo.seconds
    horas = '0' + str(int((diferenca_tempo_s/60)/60)) #capturando horas
    minutos = str(int((diferenca_tempo_s/60)%60))  #capturando minutos
    
    if len(minutos) == 1:
        minutos = '0' + minutos
    
    df['DURAÇÃO'][i] = horas + ':' + minutos
    
df['DURAÇÃO'][df['DURAÇÃO'] == '00:00'] = '06:00'    

df['DURAÇÃO'].unique()

array(['02:30', '01:00', '03:00', '03:30', '04:00', '04:30', '01:30',
       '02:00', '06:00', '00:30', '05:00', '05:30'], dtype=object)

In [31]:
df[['mensagem','hora','ENTRADA','SAIDA','DURAÇÃO']][df['DURAÇÃO'] == '06:00']

,mensagem,hora,ENTRADA,SAIDA,DURAÇÃO
16,29/05 ITAIM - MAIELY SAÍDA 18:45 ( DESLOCAD...,19:00,19:00,19:00,06:00
20,29/05 MORUMBI PELLICCIOTTI SAÍDA 13:00,13:00,07:00,13:00,06:00
101,30/05 SANTA - DIOGO SAÍDA 12:55,13:00,07:00,13:00,06:00
103,30/05 SANTA - CÔRTES SAÍDA 12:55,13:00,07:00,13:00,06:00
104,30/05 JABAQUARA - BOSI SAÍDA 13:00,13:00,07:00,13:00,06:00
105,30/05 JABAQUARA JULIANO TOBAL SAÍDA 13:00,13:00,07:00,13:00,06:00
108,30/05 JABAQUARA VIANA SAIDA 13:00,13:00,07:00,13:00,06:00
113,30/05 ITAIM - BARBOSA SAÍDA 12:45,13:00,07:00,13:00,06:00
171,31/05 ITAIM - GUILHERME SAÍDA 7:00 ( PÓS PL...,07:00,07:00,07:00,06:00
183,31/05 OSASCO - TORRELIO ENTRADA 07:00 ( DES...,07:00,07:00,13:00,06:00


In [32]:
df['setor'] = ''

for i in range(0, len(df)):
    if df['sem_ou_fds'][i] == 'semana':
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'SEMANA - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'SEMANA - MANHA - C.CIRURGICO'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'SEMANA - TARDE - C.CIRURGICO'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'SEMANA - NOTURNO - SAIDA'
        else:
            df['setor'][i] = 'VERIFICAR'

    elif df['sem_ou_fds'][i] == 'fds': 
        if df['PERIODO'][i] == '6h':
            df['setor'][i] = 'FDS - 6HS'
        elif df['PERIODO'][i] == 'manha':
            df['setor'][i] = 'FDS - C.CIRURGICO ROTINA'
        elif df['PERIODO'][i] == 'tarde':
            df['setor'][i] = 'FDS - SAIDA'
        elif df['PERIODO'][i] == 'noturno saida':
            df['setor'][i] = 'FDS - PLANTAO'
        else:
            df['setor'][i] = 'VERIFICAR'  

    else:
        df['setor'][i] = 'VERIFICAR'
        
count_freq(df, 'setor')

,Freq,Perc
setor,,
SEMANA - TARDE - C.CIRURGICO,71,31.555556
SEMANA - MANHA - C.CIRURGICO,56,24.888889
SEMANA - 6HS,54,24.000000
SEMANA - NOTURNO - SAIDA,43,19.111111
FDS - PLANTAO,1,0.444444


In [33]:
print("probs:", len(df_prob))
print("ok:", len(df))

porcentagem = round((len(df)/(len(df)+len(df_prob)))*100, 2)
print('Porcentagem de registros corrigidas pelo o algoritmo:', porcentagem)

probs: 36
ok: 225
Porcentagem de registros corrigidas pelo o algoritmo: 86.21


# Ultimos ajustes:

# Distancias

In [34]:
for i in range(0, len(df)):
    mensagem = df['mensagem'][i].upper()
    regex_D = re.findall('D[0-9]', mensagem)
    
    if len(regex_D) == 0:
        continue
    else:
        df['controle'][i] == 'DISTANCIAS'

In [35]:
for i in range(0, len(df)):
    if df['DURAÇÃO'][i] == "00:00":
        df['controle'][i] = "DISTANCIAS"
        
count_freq(df, 'controle')

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'DISTANCIAS']])
print("probs:", len(df_prob))
sample_prob('DISTANCIAS')

df = df[df['controle'] != 'DISTANCIAS']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,12,100.0


probs: 36
ok: 225


In [36]:
# Cria uma nova Série sem as listas
serie_sem_listas = pd.Series([x for x in df['unidade'] if not isinstance(x, list)])

# Imprime os valores únicos da nova Série
print(serie_sem_listas.unique())

['JABA' 'MORUMBI' 'ANALIA' 'VNS' 'VILLA' 'OSASCO' 'ITAIM' 'ALPHA' 'SANTA'
 'AVICCENA']


In [37]:
# Cria uma lista com apenas os valores que são listas
listas = [x for x in df['unidade'] if isinstance(x, list)]

# Converte as listas em tuplas e adiciona elas a um conjunto para obter apenas as listas distintas
listas_distintas = set(tuple(lista) for lista in listas)

# Imprime as listas distintas
for lista in listas_distintas:
    print(list(lista))


['ITAIM', 'MORUMBI']
['OSASCO', 'ITAIM']
['JABA', 'VILLA']
['ANALIA', 'JABA']
['OSASCO', 'JABA']
['ITAIM', 'VNS']
['MORUMBI', 'ITAIM']


In [38]:
df['setor'] = df['setor'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))

#problemas com santa isabel 
#df['setor'][(df['unidade'] == "SANTA ISABEL") & (df['setor'] == "FDS - SAIDA")] = "FDS - NOTURNO - SAIDA"
df['setor'][(df['unidade'] == "SANTA ISABEL") & (df['setor'] == "FDS - C.CIRURGICO ROTINA")] = "FDS - MANHA - C.CIRURGICO"

In [39]:
df['nome'] = df['nome'].apply(lambda x: x.replace('  ',' '))

#df['UNIDADE'] = df['UNIDADE'].apply(lambda x: x.replace('JABAQUARAQUARA','JABAQUARA'))
#df['UNIDADE'].unique()

# Salvando:

In [40]:
df['periodo'] = np.nan
df['periodo'][df['PERIODO'] == '6h'] = 1
df['periodo'][df['PERIODO'] == 'manha'] = 2
df['periodo'][df['PERIODO'] == 'tarde'] = 3
df['periodo'][df['PERIODO'] == 'noturno saida'] = 4

In [41]:
df = df[['chat', 'dia', 'nome', 'mensagem', 'controle', 'data', 'unidade',
       'extra', 'tipo', 'ENTRADA', 'SAIDA', 'DURAÇÃO', 'PERIODO','periodo',
         'setor', 'sem_ou_fds']]

In [42]:
df.head()

,chat,dia,nome,mensagem,controle,data,unidade,extra,tipo,ENTRADA,SAIDA,DURAÇÃO,PERIODO,periodo,setor,sem_ou_fds
0,[29/05/2024 15:05:22] Thales Luvisotto Neder: ...,29/05/2024,THALES LUVISOTTO NEDER,29/05 JABAQUARA - THALES SAÍDA 15:05,NaN,29/05/2024,JABA,NaN,SAIDA,13:00,15:30,02:30,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana
1,[29/05/2024 15:48:25] Juliano Moisés Tobal: 29...,29/05/2024,JULIANO MOISES TOBAL,29/05 JABAQUARA JULIANO TOBAL SAÍDA 15:05,NaN,29/05/2024,JABA,NaN,SAIDA,13:00,15:30,02:30,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana
2,[29/05/2024 15:52:53] Thiago Barreto Marques d...,29/05/2024,THIAGO BARRETO MARQUES DA SILVA,29/05 JABAQUARA NALDO SAÍDA 14:00,NaN,29/05/2024,JABA,NaN,SAIDA,13:00,14:00,01:00,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana
3,[29/05/2024 16:00:28] Amanda Kiyoko Souza Koss...,29/05/2024,AMANDA KIYOKO SOUZA KOSSAKA,29/05 MORUMBI - AMANDA SAÍDA 16:00,NaN,29/05/2024,MORUMBI,NaN,SAIDA,13:00,16:00,03:00,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana
4,[29/05/2024 16:11:50] Helena Araujo Damasceno:...,29/05/2024,HELENA ARAUJO DAMASCENO,29/05 JABAQUARA HELENA SAÍDA 16:10,NaN,29/05/2024,JABA,NaN,SAIDA,13:00,16:30,03:30,tarde,3.0,SEMANA - TARDE - C.CIRURGICO,semana


In [43]:
count_freq(df, 'controle')

,Freq,Perc
controle,,
DESLOCADO,12,100.0


# Ultimos ajustes nos deslocados:

In [44]:
df['controle'][(df['PERIODO'] == '6h') & (df['controle'] == 'DESLOCADO')] = 'DESLOCADO 6H'
count_freq(df, 'controle')

,Freq,Perc
controle,,
DESLOCADO,11,91.666667
DESLOCADO 6H,1,8.333333


In [45]:
df['unidade2'] = df['unidade']

for i in range(0, len(df)):
    if isinstance(df.loc[i, 'unidade'], list):
        df['unidade'][i] = df['unidade'][i][0]

df['unidade'][df['unidade'] == 'VILA'] = 'VILLA'
df['unidade'][df['unidade'] == 'JABA'] = 'JABAQUARA'
df['unidade'][df['unidade'] == 'SANTA'] = 'SANTA ISABEL'

sorted(df['unidade'].unique())
count_freq(df, 'unidade')

,Freq,Perc
unidade,,
ANALIA,42,18.666667
MORUMBI,41,18.222222
JABAQUARA,34,15.111111
OSASCO,30,13.333333
VILLA,26,11.555556
ITAIM,26,11.555556
VNS,14,6.222222
SANTA ISABEL,8,3.555556
ALPHA,3,1.333333


# Salvando tudo:

In [46]:
salvando_nome_arquivo('df_gabs.xlsx', df)
print('.')
salvando_nome_arquivo('df_giov.xlsx', df_prob)

225
.
36


# DISPENSADOS:

In [47]:
df = pd.read_excel('df_giov.xlsx')[['chat','dia','nome','mensagem']]
df = df.drop_duplicates(subset=['chat'])
df = df.sort_values(['nome', 'dia'])
print(len(df))
df.sample(3)

36


,chat,dia,nome,mensagem
12,[30/05/2024 13:32:22] Fabiana Barbosa Bastos: ...,30/05/2024,FABIANA BARBOSA BASTOS,30/05 JABAQUARA - FABIANA DISPENSADO
16,[30/05/2024 19:01:30] Ricardo Giraldelli Carva...,30/05/2024,RICARDO GIRALDELLI CARVALHO LIMA,30/05 VILLA - GIRALDELLI DISPENSADO
4,[30/05/2024 09:32:12] Bruno Ferraz de Campos B...,30/05/2024,BRUNO FERRAZ DE CAMPOS BARONE,30/05 VNS BARONE DISPENSADO


In [48]:
#RETIRANDO REGISTROS DE DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE OU DISPENSA

df['controle'] = np.nan

#preparando para reconhecer corretamente

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DISPENSADO|SAUDE|DISPENSADA', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes == 0:
        df['controle'][i] = 'PROB'
        
df['controle'].value_counts() 

#salvando
df_prob = df[df['controle'] == 'PROB']
df_prob.reset_index(drop=True, inplace=True)
print("unidades probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("unidade ok:", len(df)) 

unidades probs: 2
unidade ok: 34


In [49]:
#DATA DO PLANTÃO
df['controle'] = ''
df['data'] = ''

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
        df['controle'][i] = 'NORM'
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'PROB'

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] != 'NORM']])
df_prob.reset_index(drop=True, inplace=True)
print("data probs:", len(df_prob))

df = df[df['controle'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("datas ok:", len(df)) 

df['data'] = df['data']+"/2024"
sorted(df['data'].unique())

data probs: 2
datas ok: 34


['30/05/2024', '31/05/2024', '31/5/2024']

In [50]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = ''

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio
    
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

#Retirando aqueles que contem horas no registro da mensagem
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas >= 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
        df['controle'][i] = 'PROB'
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'OK'
        
print(df['controle'].value_counts())

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] != 'OK']])
df_prob.reset_index(drop=True, inplace=True)
print("probs:", len(df_prob))

df = df[df['controle'] == 'OK']
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

controle
OK    34
Name: count, dtype: int64
probs: 2
ok: 34


In [51]:
print(len(df_prob))
print(len(df))

2
34


In [52]:
df = df.drop_duplicates(subset=['chat'])

os.remove("df_giov.xlsx")
salvando_nome_arquivo('df_giov.xlsx', df_prob)
salvando_nome_arquivo('df_dispensados.xlsx', df)

2
34


# ===========================================

In [53]:
print(len(pd.DataFrame({'chat': read_file('_chat.txt') })))
print(len(pd.read_excel('df_dispensados.xlsx')))
print(len(pd.read_excel('df_gabs.xlsx')))
print(len(pd.read_excel('df_giov.xlsx')))

261
34
225
2
